In [87]:
import numpy as np
import pandas as pd
from pathlib import Path
import sys
sys.path.append(str(Path().resolve().parent))
import PixelArrays as array

In [105]:
# Total severity score in %
total_severity_score = {
    0: (0,0),
    1: (0, 25), 
    2: (26, 50), 
    3: (51, 75),
    4: (76, 100)
}

def calculate_ratio_tts(lung_mask, infection_mask):
    lung_area = np.count_nonzero(lung_mask)
    if lung_area == 0:
        return -1, -1
    infection_area = np.count_nonzero(infection_mask)
    if infection_area == 0:
        return 0, 0
    ratio = infection_area / lung_area * 100
    tts = 0
    for score, (lower, upper) in total_severity_score.items():
        if (ratio >= lower) and (ratio <= upper):
            tts = score
            break
    
    return ratio, tts

def check_examination(lung_slices, infection_slices):
    analzyed_slices = []
    results = []
    for i in range(len(lung_slices)):
        ratio, score = calculate_ratio_tts(lung_slices[i], infection_slices[i])
        if ratio != -1:
            analzyed_slices.append(i)
            results.append([ratio, score])
    return np.column_stack((analzyed_slices, results))

def create_report_severity(lung_slices, infection_slices):
    # Total number of slices of analyzed image
    total_slices = len(lung_slices)
    result = check_examination(lung_slices, infection_slices)
    # Number of slices with nonzero area of lungmask
    analyzed_slices = result.shape[0]
    # Unique values of severity score with number of occurence
    unique, counts = np.unique(result[:,2], return_counts=True)
    unique = unique.astype('int')
    # Percentage of the occurence of score
    percentages = counts / analyzed_slices * 100
    
    report = {
        "Total slices": total_slices, 
        "Analyzed slices": analyzed_slices, 
    }
    for i in range(len(unique)):
        score = unique[i]
        report["Score {0} occurence number".format(score)] = counts[i]
        report["Score {0} percentage number %".format(score)] = percentages[i]
    
    return report

In [106]:
def create_dataframe_from_nitfi_images(folder_path, save_path=None):
    path = Path(folder_path)
    lungs_path = path / "lung_mask"
    infection_path = path / "infection_mask"

    lungs_files = [x for x in lungs_path.glob('**/*') if x.is_file()]
    infection_files = [x for x in infection_path.glob('**/*') if x.is_file()]

    df = pd.DataFrame(columns=["Image name", 
                               "Total slices", "Analyzed slices", 
                               "Score 0 occurence number", "Score 0 percentage number %", 
                               "Score 1 occurence number", "Score 1 percentage number %", 
                               "Score 2 occurence number", "Score 2 percentage number %", 
                               "Score 3 occurence number", "Score 3 percentage number %", 
                               "Score 4 occurence number", "Score 4 percentage number %", ])
    for i in range(len(lungs_files)):
        image_name = lungs_files[i].name
        lung = array.get_pixel_array_nii_all(lungs_files[i])
        infection = array.get_pixel_array_nii_all(infection_files[i])
        report = create_report_severity(lung, infection)
        report["Image name"] = image_name
        df = df.append(report, ignore_index=True)
        
    df = df.fillna(0)
    
    if save_path is not None:
        df.to_csv(save_path)
        
    return df
    

In [107]:
path = Path(r"D:\Studia\sem7\inzynierka\data\segmented_nii_kaggle")
create_dataframe_from_nitfi_images(path)

,Image name,Total slices,Analyzed slices,Score 0 occurence number,Score 0 percentage number %,Score 1 occurence number,Score 1 percentage number %,Score 2 occurence number,Score 2 percentage number %,Score 3 occurence number,Score 3 percentage number %,Score 4 occurence number,Score 4 percentage number %
0,coronacases_001.nii,301,227,66,29.074890,153,67.400881,8,3.524229,0.0,0.000000,0.0,0.000000
1,coronacases_002.nii,200,176,33,18.750000,143,81.250000,0,0.000000,0.0,0.000000,0.0,0.000000
2,coronacases_003.nii,200,150,15,10.000000,53,35.333333,64,42.666667,18.0,12.000000,0.0,0.000000
3,coronacases_004.nii,270,251,138,54.980080,113,45.019920,0,0.000000,0.0,0.000000,0.0,0.000000
4,coronacases_005.nii,290,270,154,57.037037,116,42.962963,0,0.000000,0.0,0.000000,0.0,0.000000
5,coronacases_006.nii,213,180,110,61.111111,70,38.888889,0,0.000000,0.0,0.000000,0.0,0.000000
6,coronacases_007.nii,249,219,126,57.534247,93,42.465753,0,0.000000,0.0,0.000000,0.0,0.000000
7,coronacases_008.nii,301,249,33,13.253012,216,86.746988,0,0.000000,0.0,0.000000,0.0,0.000000
8,coronacases_009.nii,256,222,111,50.000000,111,50.000000,0,0.000000,0.0,0.000000,0.0,0.000000
9,coronacases_010.nii,301,212,27,12.735849,168,79.245283,17,8.018868,0.0,0.000000,0.0,0.000000
